# COMP 4446 / 5046 Lab 5, 2024

Based on:
- ["Text classification from scratch"](https://keras.io/examples/nlp/text_classification_from_scratch/) by Mark Omernick, Francois Chollet
- ["Bidirectional LSTM on IMDB"](https://keras.io/examples/nlp/bidirectional_lstm_imdb/) by Francois Chollet
- ["GPT2 Text Generation with KerasNLP"](https://keras.io/examples/generative/gpt2_text_generation_with_kerasnlp/) by Chen Qian
- ["GPT text generation from scratch with KerasNLP"](https://keras.io/examples/generative/text_generation_gpt/) by Jesse Chan

Make sure you are using a GPU for this lab (otherwise some of the later steps will take a long time).

## Introduction

In this lab, we will revisit classifiers and RNNs, but using the Keras library, which is built on top of Tensorflow (rather than PyTorch).

We'll see two examples of getting data and making a model to do movie review prediction on IMDB data.

First, we'll work from raw text - a set of text files on disk.

## Setup

In [51]:
!pip install git+https://github.com/keras-team/keras-nlp.git -q
!pip install tensorflow

In [1]:
import os
import time

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
import tensorflow as tf
import numpy as np
from keras import layers
import keras_nlp

2024-03-19 00:57:17.795704: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load the data: IMDB movie review sentiment classification

Let's download the data and inspect its structure.

In [5]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  19.4M      0  0:00:04  0:00:04 --:--:-- 19.7M


The `aclImdb` folder contains a `train` and `test` subfolder, plus a few text files:

In [4]:
!ls aclImdb

README	imdb.vocab  imdbEr.txt	test  train


Inside each folder are subfolders for positive and negative data, plus files with precomputed features, and source URLs.

In [5]:
!ls aclImdb/test

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


In [6]:
!ls aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


The `aclImdb/train/pos` and `aclImdb/train/neg` folders contain text files, each of
 which represents one review (either positive or negative):

In [7]:
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

We are only interested in the `pos` and `neg` subfolders, so let's delete the other subfolder that has text files in it:

In [8]:
!rm -r aclImdb/train/unsup

You can use the utility `keras.utils.text_dataset_from_directory` to
generate a labeled `tf.data.Dataset` object from a set of text files on disk filed into class-specific folders.

Let's use it to generate the training, validation, and test datasets. The validation and training datasets are generated from two subsets of the `train` directory, with 20% of samples going to the validation dataset and 80% going to the training dataset. When using the `validation_split` & `subset` arguments, make sure to either specify a random seed, or to pass `shuffle=False`, so that the validation & training splits you get have no overlap.

In [9]:
batch_size = 32
raw_train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


2024-03-19 00:10:07.622990: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


Let's look at a few samples:

In [10]:
# It's important to take a look at your raw data to ensure your normalization
# and tokenization will work as expected. We can do that by taking a few
# examples from the training set and looking at them.
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

2024-03-19 00:10:37.841613: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Prepare the data

We will use the `TextVectorization` layer for word splitting & indexing. In the process, we will also remove `<br />` tags.

In [11]:
import string
import re


# Having looked at our data above, we see that the raw text contains HTML break
# tags of the form '<br />'. These tags will not be removed by the default
# standardizer (which doesn't strip HTML). Because of this, we will need to
# create a custom standardization function.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Now that the vectorize_layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)

2024-03-19 00:11:13.324959: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Two options to vectorize the data

There are 2 ways we can use our text vectorization layer:

**Option 1: Make it part of the model**, so as to obtain a model that processes raw strings, like this:

```python
text_input = keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
x = layers.Embedding(max_features + 1, embedding_dim)(x)
...
```

**Option 2: Apply it to the text dataset** to obtain a dataset of word indices, then feed it into a model that expects integer sequences as inputs.

An important difference between the two is that option 2 enables you to do
**asynchronous CPU processing and buffering** of your data when training on a GPU. So if you're training a model on a GPU, you probably want to go with this option to get the best performance. This is what we will do below.

If we were to export our model to production, we'd ship a model that accepts rawstrings as input, like in the code snippet for option 1 above. This can be done after training. We do this in the last section.

In [12]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

## Build a model

The first model we introduce uses a Convolutional Neural Network. This has not been covered in lectures (you can read more about it on [Wikipedia](https://en.wikipedia.org/wiki/Convolutional_neural_network#Building_blocks)). The general idea is the a convolution is a type of weighted average that is applied in multiple locations.

In [13]:
# A integer input for vocab indices.
inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a hidden layer (linear + ReLU):
x = layers.Dense(128, activation="relu")(x)
# Dropout makes some values 0 during training, which tends to help improve how well the model generalised
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## Train the model

We'll run just 3 epochs / iterations so you can see how it works. Later, if you have more time you can run it for longer to see how well it can do.

In [14]:
epochs = 3

# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 36s 55ms/step - accuracy: 0.5852 - loss: 0.6222 - val_accuracy: 0.8670 - val_loss: 0.3191
Epoch 2/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 33s 53ms/step - accuracy: 0.8842 - loss: 0.2873 - val_accuracy: 0.8772 - val_loss: 0.3087
Epoch 3/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 33s 53ms/step - accuracy: 0.9447 - loss: 0.1477 - val_accuracy: 0.8434 - val_loss: 0.4708


## Evaluate the model on the test set

We also evaluate on the test data. The accuracy is lower than on the training data because the test data is unseen.

In [15]:
model.evaluate(test_ds)

782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8375 - loss: 0.4655


[0.46363094449043274, 0.8391600251197815]

## Make an end-to-end model

If you want to obtain a model capable of processing raw strings, you can create a new model using the weights we just trained, but a different input type and an extra vectorisation step:

In [16]:
# A string input
inputs = keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# Our end to end model
end_to_end_model = keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8375 - loss: 0.4673


[0.4634109139442444, 0.8391600251197815]

# LSTM

Now let's see how to construct an LSTM in Keras.

First, we'll set up some parameters:

In [17]:
import numpy as np
import keras
from keras import layers

max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review

Keras has code for the LSTM already, so we don't need to implement all the details ourselves.

The code below sets up the model, using integers as input (ie., like at the start above, we assume someone else is converting our input words into word IDs):

In [40]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)

# Add a bidirectional LSTM
# return_sequences determines whether we get just the last vector or all:
#    True - give us an output for each word
#    False - give us just one output
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=False))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_18 (InputLayer)     │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_23 (Embedding)        │ (None, None, 128)      │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_23                │ (None, None, 128)      │        98,816 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_24                │ (None, 128)            │        98,816 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,757,761 (10.52 MB)

 Trainable params: 2,757,761 (10.52 MB)

 Non-trainable params: 0 (0.00 B)

Load the IMDB movie review sentiment data. This time we are going to use the library provided version of the same data.

In [36]:
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(
    num_words=max_features
)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
# Use pad_sequence to standardize sequence length:
# this will truncate sequences longer than 200 words and zero-pad sequences shorter than 200 words.
x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)

25000 Training sequences
25000 Validation sequences


Train and evaluate the model. You will see similar results to the approach above. To do better, we would need to train for longer and explore other variations on the model structure.

In [37]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/keras/src/optimizers/base_optimizer.py:576: UserWarning: Gradients do not exist for variables ['embeddings', 'kernel', 'recurrent_kernel', 'bias', 'kernel', 'recurrent_kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.5052 - loss: 0.6938

KeyboardInterrupt: 

# Task 1 - Multilayer LSTM

Modify the LSTM architecture to have two layers. You can do this by having an extra layer that produces output at each position, which is fed into the next layer as an input.

You should print the model summary to show for credit.

In [41]:
# TODO
model.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_18 (InputLayer)     │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_23 (Embedding)        │ (None, None, 128)      │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_23                │ (None, None, 128)      │        98,816 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_24                │ (None, 128)            │        98,816 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,757,761 (10.52 MB)

 Trainable params: 2,757,761 (10.52 MB)

 Non-trainable params: 0 (0.00 B)

# Text generation

Now, we are going to turn to text generation to see some variations on inference.

You will learn to load a pre-trained  Language Model (LM) - [GPT-2](https://openai.com/research/better-language-models) (originally invented by OpenAI), finetune it to a specific text style, and generate text based on users' input (also known as prompt). You will also see how GPT2 adapts quickly to non-English languages, such as Chinese.

This examples uses [Keras Core](https://keras.io/keras_core/) to work in any of `"tensorflow"`, `"jax"` or `"torch"`. Support for Keras Core is baked into KerasNLP, simply change the `"KERAS_BACKEND"` environment variable to select the backend of your choice. We select the JAX backend below.

In [6]:
os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

keras.mixed_precision.set_global_policy("mixed_float16")

Large Language Models are complex to build and expensive to train from scratch. Luckily there are pretrained LLMs available for use right away. Keras provides a large number of pre-trained checkpoints that allow you to experiment with very good models without needing to train them yourself.

Keras also  has a Sampler class that implements generation algorithms such as Top-K, Beam and contrastive search. These samplers can be used to generate text with custom models.

## Load a pre-trained GPT-2 model and generate some text

KerasNLP provides a number of pre-trained models, such as [Google
Bert](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html)
and [GPT-2](https://openai.com/research/better-language-models). You can see
the list of models available in the [KerasNLP repository](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/models).

It's very easy to load the GPT-2 model as you can see below:

In [2]:
# To speed up training and generation, we use preprocessor of length 128
# instead of full length 1024.
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

2024-03-19 00:57:45.006773: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Once the model is loaded, you can use it to generate some text right away. Run
the cells below to give it a try. It's as simple as calling a single function
*generate()*:

In [44]:
start = time.time()

output = gpt2_lm.generate("My trip to Yosemite was", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")

2024-03-19 00:33:47.767248: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
I0000 00:00:1710808431.081823     167 service.cc:145] XLA service 0x561aa27cb8d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1710808431.081860     167 service.cc:153]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1710808442.679341     167 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



GPT-2 output:
My trip to Yosemite was an amazing experience. It was a huge, beautiful place, and it was a great experience to go out to the park. It was a very cool place to be at the end of a day. I went for a walk and a walk and then I got to see the Yosemite Valley.

There were some really cool places to be in the park and the park was a really nice place to spend the day, especially when you were just starting out on the hike. I really enjoyed going down the valley. The trail was really steep and I didn't really have a lot of time to hike up to it.

After my first day, I went back down the mountain and I really enjoyed my time in the park. I didn't think I'd be back for another day, but I'm really glad I did. I love it here and I'm going to go back for another day.

I love Yosemite!

I'm going to
TOTAL TIME ELAPSED: 25.83s


Try another one:

In [45]:
start = time.time()

output = gpt2_lm.generate("That Italian restaurant is", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
That Italian restaurant is known as the "Italian Restaurant" in Italy. It was named the "French Restaurant" after its owner who served it for over 40 years. And the restaurant's name has become synonymous with Italian cuisine.

Italian Restaurant, located in the heart of Milan, is known for the unique Italian style of food, which is made from the same ingredients as the French.

"It is very easy to make a French restaurant," says chef Marco Mancini. "It is very simple and easy to make. And it is also very delicious. We make a lot of pasta, we make a lot of meatballs, we make a lot of bread. So you can make this very delicious and very tasty. It is very easy to make."

And, of course, the food is also made from a whole bunch of ingredients, including fresh herbs.

"We use fresh herbs, we use fresh vegetables, we use fresh fruits," says Manc
TOTAL TIME ELAPSED: 10.62s


Notice how much faster the second call is. This is because the computational
graph is [XLA compiled](https://www.tensorflow.org/xla) in the 1st run and
re-used in the 2nd behind the scenes.

The quality of the generated text looks OK, but we can improve it via
fine-tuning.

## More on the GPT-2 model from KerasNLP

Next up, we will actually fine-tune the model to update its parameters, but
before we do, let's take a look at the full set of tools we have to for working
with for GPT2.

The code of GPT2 can be found
[here](https://github.com/keras-team/keras-nlp/blob/master/keras_nlp/models/gpt2/).
Conceptually the `GPT2CausalLM` can be hierarchically broken down into several
modules in KerasNLP, all of which have a *from_preset()* function that loads a
pretrained model:

- `keras_nlp.models.GPT2Tokenizer`: The tokenizer used by GPT2 model, which is a
    [byte-pair encoder](https://huggingface.co/course/chapter6/5?fw=pt).
- `keras_nlp.models.GPT2CausalLMPreprocessor`: the preprocessor used by GPT2
    causal LM training. It does the tokenization along with other preprocessing
    works such as creating the label and appending the end token.
- `keras_nlp.models.GPT2Backbone`: the GPT2 model, which is a stack of
    `keras_nlp.layers.TransformerDecoder`. This is usually just referred as
    `GPT2`.
- `keras_nlp.models.GPT2CausalLM`: wraps `GPT2Backbone`, it multiplies the
    output of `GPT2Backbone` by embedding matrix to generate logits over
    vocab tokens.

## Finetune on News data

Now you have the knowledge of the GPT-2 model from KerasNLP, you can take one step further to finetune the model so that it generates text in a specific style, short or long, strict or casual. In this tutorial, we will use a news dataset.

In [54]:
!pip install tensorflow_datasets

     |████████████████████████████████| 5.0 MB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 111.4 MB/s eta 0:00:01
     |████████████████████████████████| 140 kB 122.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 76.3 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 112.7 MB/s eta 0:00:01
     |████████████████████████████████| 229 kB 103.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 64.1 MB/s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21483 sha256=2d1dcffb0da7b51c55e3a6cefd77513b51969b22522a1f0fd71fae1a77a74457
  Stored in directory: /home/studio-lab-user/.cache/pip/wheels/e1/e8/83/ddea66100678d139b14bc87692ece57c6a2a937956d2532608
Successfully built promise
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled

In [3]:
import tensorflow_datasets as tfds

news_ds = tfds.load("ag_news_subset", split="train", as_supervised=True)

Let's take a look inside sample data from the ag_news TensorFlow Dataset. Thereare two features:

- **__description__**: text of the article.
- **__label__**: the category (used in earlier labs).

In [4]:
for description, label in news_ds:
    print(description.numpy())
    print(label.numpy())
    break

b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
3


2024-03-19 00:58:03.315034: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In our case, we are performing next word prediction in a language model, so we
only need the 'description' feature.

In [5]:
train_ds = (
    news_ds.map(lambda description, _: description)
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

Now you can finetune the model using the familiar *fit()* function. Note that
`preprocessor` will be automatically called inside `fit` method since
`GPT2CausalLM` is a `keras_nlp.models.Task` instance.

This step takes quite a bit of GPU memory and a long time if we were to train
it all the way to a fully trained state. Here we just use part of the dataset
for demo purposes.

In [ ]:
train_ds = train_ds.take(10)
num_epochs = 1

# Linearly decaying learning rate.
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.5,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs)

I0000 00:00:1710809436.899502    1135 service.cc:145] XLA service 0x7fdc3c01f990 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1710809436.899538    1135 service.cc:153]   StreamExecutor device (0): Host, Default Version
W0000 00:00:1710809438.444595    1135 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


After fine-tuning is finished, you can again generate text using the same
*generate()* function. This time, the text will be closer to news writing
style, and the generated length will be close to our preset length in the
training set.

In [6]:
start = time.time()

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")

2024-03-19 00:58:12.992566: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
I0000 00:00:1710809896.301220    1413 service.cc:145] XLA service 0x559c0de36310 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1710809896.301253    1413 service.cc:153]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1710809903.680002    1413 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



GPT-2 output:
I like basketball because it's fun. It's not about the game, but about winning. I like to watch it because I think it's so much fun. I love the game of basketball. I think the game is so important to me and I like to watch it because it's so fun. I love the game of basketball.

I'm a big fan of basketball, and I like to watch it because of the way it's done. The games have been played so many times, and I'm so excited to watch it. I think the NBA has changed a lot since the days of the NBA, and I'm so happy to watch it because it's so fun. I like to watch it because it's so fun. The games have been played so many times, and I'm so excited to watch it.

I'm a big fan of basketball. I like to watch it because it's so fun. I love the game of basketball. I think the
TOTAL TIME ELAPSED: 38.42s


## Into the Sampling Method

In KerasNLP, there are a few sampling methods, e.g., contrastive search,
Top-K and beam sampling. By default, our `GPT2CausalLM` uses Top-k search, but you can choose your own sampling method.

Much like optimizer and activations, there are two ways to specify your custom sampler:

- Use a string identifier, such as "greedy", you are using the default
configuration via this way.
- Pass a `keras_nlp.samplers.Sampler` instance, you can use custom configuration via this way.

In [ ]:
# Use a string identifier.
gpt2_lm.compile(sampler="top_k")
output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)

# Use a `Sampler` instance. `GreedySampler` tends to repeat itself,
greedy_sampler = keras_nlp.samplers.GreedySampler()
gpt2_lm.compile(sampler=greedy_sampler)

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)

# Task 2 - Varying Samplers

Try several other samplers:

- `BeamSampler`, keeps track of the `num_beams` most probable sequences at each timestep, and predicts the best next token from all sequences.
- `ContrastiveSampler`, reweights options based on their similarity to previously generated outputs (to decrease repetition).
- `RandomSampler`, at each time step, samples the next token using the softmax probabilities provided by the model.
- `TopKSampler`, select the top`k` most probable tokens and then sample from them.
- `TopPSampler`, like Top-K, except it selects enough samples to cover `P` percent of the distribution (which could be more or less than the `k` value from the previous approach).

For more information about these, see: https://keras.io/api/keras_nlp/samplers/

To complete the task, show that you have implemented all five and used them to generate output.

You should experiment with the parameters for each sampler and see what patterns you observe in terms of the outputs (this is not marked).

In [7]:
# Use Beam
sampler = keras_nlp.samplers.BeamSampler(num_beams=5)
gpt2_lm.compile(sampler=sampler)

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)


GPT-2 output:
I like basketball. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to


In [13]:
# Use Beam
sampler = keras_nlp.samplers.BeamSampler(num_beams=100)
gpt2_lm.compile(sampler=sampler)

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)


GPT-2 output:
I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball. I like basketball


In [14]:
# Use Contrastive
sampler = keras_nlp.samplers.ContrastiveSampler(k=10)
gpt2_lm.compile(sampler=sampler)

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)


GPT-2 output:
I like basketball. I like to play it. I like to watch it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I like to listen to it. I


In [9]:
# Use Random
sampler = keras_nlp.samplers.RandomSampler(temperature=0.7)
gpt2_lm.compile(sampler=sampler)

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)


GPT-2 output:
I like basketball, but I also like to run. I think I'm a little bit too small, so I'm probably not fit for the NBA. I think I'm too big. I'm kind of a stock-a-pony. I can play basketball if I want to. I'm not doing that. I don't want to be the guy who can't play basketball. So you know, I'm just trying to make the best of my opportunity.

"I've been doing it since I was 9. I'm still a little bit young, but I'm going to continue to learn and move forward."

The only problem is that he is recently in a slump.

"I'm not getting better," he said. "I kind of looked up to the NBA and I was just trying to make the best of it. But I guess I've come a long way. I've seen the way I played the game. I'm just trying


In [10]:
# Use Top K
sampler = keras_nlp.samplers.TopKSampler(k=5, temperature=0.7)
gpt2_lm.compile(sampler=sampler)

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)


GPT-2 output:
I like basketball. I like playing with a lot more energy, more intensity, more effort, more grit, more effort and more determination. I like to be the best player I can be and be able to win championships.

The only problem is that I'm not good enough to be a great player.

The only problem is that I'm not good enough to be a great player. I'm not good enough to play well enough to be a good player. I'm not good enough to be a good player to be successful, to be an NBA All-Star.

I'm not good enough to play well enough to be a great athlete to be successful. I'm not good enough to play well enough to be a great player to be successful to be successful to be successful to be successful to be successful to be successful to be successful to be successful to be successful to be successful to be successful to be successful to be successful to be successful to be successful to be


In [16]:
# Use Top P
sampler = keras_nlp.samplers.TopPSampler(p=0.8)
gpt2_lm.compile(sampler=sampler)

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)


GPT-2 output:
I like basketball but I have a love of hockey and you can't expect me to spend more than a few minutes on the court or my friend Blake Hines is my goalie. I'm also a fan of Sunday Night Football and was part of the Cincinnati Bengals roster at the beginning of the 2013-14 season.

However, I'm now home and don't really have much to be proud of, so my family and friends are going to come to your house. It's going to be my first time meeting my new girlfriend and I just want to be happy and to feel comfortable.

It's hard to sit there and have people say to you "Oh my God, how are you doing?" and then turn around and look at you and say "Oh my god, I've never played hockey before!"

Nowadays, I'll try to keep my "wow factor" at 3 years old. I'm gonna try to stick with what's been through in my life



# [optional] Finetune on Chinese Poem Dataset

We can also finetune GPT2 on non-English datasets. For readers knowing Chinese,
this part illustrates how to fine-tune GPT2 on Chinese poem dataset to teach our
model to become a poet!

Because GPT2 uses byte-pair encoder, and the original pretraining dataset
contains some Chinese characters, we can use the original vocab to finetune on
Chinese dataset.

In [ ]:
!# Load chinese poetry dataset.
!git clone https://github.com/chinese-poetry/chinese-poetry.git

Load text from the json file. We only use (全唐诗 - "All Tang poetry") for demo purposes.

In [ ]:
import os
import json

poem_collection = []
for file in os.listdir("chinese-poetry/全唐诗"):
    if ".json" not in file or "poet" not in file:
        continue
    full_filename = "%s/%s" % ("chinese-poetry/全唐诗", file)
    with open(full_filename, "r") as f:
        content = json.load(f)
        poem_collection.extend(content)

paragraphs = ["".join(data["paragraphs"]) for data in poem_collection]

Let's take a look at sample data.

In [ ]:
print(paragraphs[0])

This is translated by Google Translate as "Stay in Qingjidian, Chaoyang Lidi City. In the good years, people enjoy their work and sing songs on the ridges."

Similar to the news example, we convert to TF dataset, and only use partial data to train.

In [ ]:
train_ds = (
    tf.data.Dataset.from_tensor_slices(paragraphs)
    .batch(16)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

# Running through the whole dataset takes long, only take `500` and run 1
# epochs for demo purposes.
train_ds = train_ds.take(500)
num_epochs = 1

learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-4,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs)

Let's check the result!

In [ ]:
output = gpt2_lm.generate("昨夜雨疏风骤", max_length=200) # Input is "Rainy and windy last night"
print(output)